#### 네이버 뉴스 크롤링
##### 2020-01~2020-10
##### 기사날짜 이른 순으로 정렬, 기사날짜 

In [1]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

In [2]:
date = str(datetime.now())
date = date[: date.rfind(':')].replace(' ', '_')
date = date.replace(':', '시') + '분'

In [7]:
#예시로 구현.. 전체 몇개인지 카운트 필요..?
query = input('검색 키워드를 입력하세요 : ')
query = query.replace('', '+')
news_num = int(input('총 필요한 뉴스기사를 입력해주세요(숫자만 입력) : '))

검색 키워드를 입력하세요 : 홈트
총 필요한 뉴스기사를 입력해주세요(숫자만 입력) : 100


In [8]:
news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
req = requests.get(news_url.format(query))
soup = BeautifulSoup(req.text, 'html.parser')

In [9]:
news_dict = {}
idx = 0 
cur_page = 1 #뉴스 갯수가 더 많으면 다음 페이지로 넘어가야 하니까 기억해둘것

In [11]:
print()
print('크롤링 !!!!!!! 성공했으면 좋겠어..무척...')

while idx < news_num:
    
    table = soup.find('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
    
    for n in a_list[:min(len(a_list), news_num-idx)]:
        news_dict[idx] = {'title' : n.get('title'),
                          'url' : n.get('href') }
        idx += 1

    cur_page += 1
    
    pages = soup.find('div', {'class' : 'sc_page_inner'})
    next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')
    
    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')


크롤링 중...


In [14]:
#일단 엑셀로 뽑아보까.. csv로 ..?
folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_{}_{}.xlsx'.format(query, date)
news_df.to_excel(xlsx_file_name)

###### 아니 이거 페이지 기간설정 어떻게..?
###### 실제 입력해야되는데..? 아예 설정된 페이지를 가져와서 데이터 추출?

In [16]:
import time

In [19]:
url = 'https://search.naver.com/search.naver?where=news&query=%ED%99%88%ED%8A%B8&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2020.01.01&de=2020.10.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20200101to20201031&is_sug_officeid=0'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')

In [20]:
#기사제목
soup.find_all('a', attrs={'class':'news_tit'})
[title['title'] for title in soup.find_all('a', attrs={'class':'news_tit'}) ]

["[봉쥬르 2020] 올해 주목해야 할 '5G·AI·구독경제'",
 "[오늘의 책]금연·다이어트…새해에도 작심삼일? '습관의 완성'",
 "주말의 홈트ㅣ한수진의 요가 한 수, '잠자고있는 복근 깨우는 요가'",
 '`괴팍한5형제` 방법에 대해 묻자, 김종국은 운동 얘기라면 한 회로는 모자란데라며 미소를 띄어!',
 '\'괴팍한5형제\' 김종국 "빨 팬티 입고 운동하면 돼"...서장훈 기겁 \'폭소\'',
 '다이어트 운동 필라테스, 폼롤러와 함께하는 동작들',
 '소소하지만 확실한 홈트',
 "르꼬끄, 새해 몸만들기 '르스타트 2020' 캠페인 진행",
 '코카콜라 박보영 반신욕&홈트 광고 촬영 공개',
 '르꼬끄 스포르티브 ‘르스타트(LE START) 2020’ 캠페인 전개']

In [22]:
#기사주소
[ url['href'] for url in soup.find_all('a', attrs={'class':'news_tit'}) ]

#기사작성일-태그중복, 이상한거 가져오는데...?-정규표현식 정리?
soup.find_all('span', attrs={'class':'info'})

[<span class="info">2020.01.01.</span>,
 <span class="info">2020.01.01.</span>,
 <span class="info">2020.01.01.</span>,
 <span class="info">2020.01.01.</span>,
 <span class="info">2020.01.02.</span>,
 <span class="info">2020.01.02.</span>,
 <span class="info">2020.01.02.</span>,
 <span class="info">2020.01.02.</span>,
 <span class="info">2020.01.02.</span>,
 <span class="info">2020.01.02.</span>]

##### 이상한 거 다 버리고 날짜만 가져와보자꾸나

In [23]:
dates = [ date.get_text() for date in soup.find_all('span', attrs={'class':'info'})]

import re

date_list = []
for date in dates:
    if re.search(r'\d+.\d+.\d+.', date) != None:
        date_list.append(date)

date_list

['2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.02.',
 '2020.01.02.',
 '2020.01.02.',
 '2020.01.02.',
 '2020.01.02.',
 '2020.01.02.']

In [ ]:
import pandas as pd
start = 1
result_df = pd.DataFrame()

while True:
    try:
        url = 'https://search.naver.com/search.naver?where=news&query=%ED%99%88%ED%8A%B8&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2020.01.01&de=2020.10.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20200101to20201031&is_sug_officeid=0'.format(start)
        headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        news_title = [title['title'] for title in soup.find_all('a', attrs={'class':'news_tit'})] # 기사 제목
        news_url = [ url['href'] for url in soup.find_all('a', attrs={'class':'news_tit'}) ] # 기사 url
        
        dates = [ date.get_text() for date in soup.find_all('span', attrs={'class':'info'})] # 기사 작성일
        news_date = []
        for date in dates:
            if re.search(r'\d+.\d+.\d+.', date) != None: # 기사 작성일 정제
                news_date.append(date)
        
        df = pd.DataFrame({'기사작성일':news_date,'기사제목':news_title,'기사주소':news_url})
        result_df = pd.concat([result_df, df], ignore_index=True)
        start += 10
    
    except: # 오류발생시 몇 페이지까지 크롤링했는지 page를 확인하기 
        print(start)
        break

In [ ]:
result_df